### RULES
Requires format of skipped line between different phylums (ex. empty row above Diatom, Dinoflagellate, etc.)

Assumed all {Ochromonas, } are mixotrophs.

1. assume everything after "Unknown flagellates" is irrelevant (to be deleted)
2. diatoms are NOT mixotrophs
3. remove all "[name]-like" (without genus specified)
4. remove all "[genus name] spp." AND "[genus name] sp."
5. check "cysts of"

Status Key--  
Confirmed := explicitly in the Mixotroph Database  
Unsure (sp. in mdb) := genus in Mixotroph Database lists "[genus name] sp." (ex. Ochromonas sp. for Ochromonas danica)  
Unsure (inexact name):= LIS name is in a longer Mixotroph Database name or vice versa (ex. Chattonella marina in Chattonella marina var. ovata)   

### QUESTIONS TO ASK

1. Should I be considering "cysts of Linggulodinium polyedrum" mixotrophs?
2. How should I handle these "unsure" cases? - see status key above

In [60]:
import pandas as pd
import numpy as np

from datetime import datetime

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option("future.no_silent_downcasting", True)

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [61]:
mdb = pd.read_csv("MDB - 3Dec2022.csv")
mdb.columns = mdb.iloc[1]
mdb = mdb.drop([0, 1]).reset_index(drop=True)

# edit mdb so that species ending in "sp" now end in "sp."
mdb['Species Name'] = mdb['Species Name'].str.replace(r'sp$', 'sp.', regex=True)

mdb.head()

1,Species Name,Taxonomic Group,AphiaID,Additional notes,Gene markers,PR2 Accession Number,GenBank Accession Number,Reference to sequence,MFT,Evidence of mixoplankton activity,Reference to mixoplankton activity,can form colonies,has benthic stages,has parasitic stages,HABs,L (μm),W (μm) or diameter (μm),Depth (μm) or other information,size class,colony size,Reference to mixoplankton size,ability to use NO3,Reference to NO3 usage,Mode of feeding,Presumed prey size,tested min size (μm),tested max size (μm),ingested min size (μm),ingested max size (μm),# prey spp tested,Foram ontogenetic stage (20 μm),presumed prey size class for col AG,Foram ontogenetic stage (20-200 μm),Presumed prey size (µm) for col AI,Foram ontogenetic stage (200 μm - adult),Presumed prey size (µm) for col AK,References for prey sizes,example prey types,prey taxonomic group,References for prey types,Plastid source,plastid taxonomic group,indicative size in environment,References for plastids,Symbiont source,Symbiont taxonomic group,References for taxonomic information,symbiont size (environment),symbiont size (in hospite),indicative size,symbionts per host,References and additional notes for symbionts,NaN,ALSK,ANTA,APLR,ARAB,ARCH,ARCT,AUSE,AUSW,BENG,BERS,BPLR,BRAZ,CAMR,CARB,CCAL,CHIL,CHIN,CNRY,EAFR,ETRA,FKLD,GFST,GUIA,GUIN,INDE,INDW,ISSG,KURO,MEDI,MONS,NADR,NASE,NASW,NATR,NECS,NEWZ,NPPF,NPSW,NPTG,NWCS,PEQD,PNEC,PSAE,PSAW,REDS,SANT,SARC,SATL,SPSG,SSTC,SUND,TASM,WARM,WTRA
0,Acanthochiasma sp.,Radiolaria,368427,Acantharia,18S_rRNA_nucleus;18S_rRNA_nucleus;18S_rRNA_nuc...,HM103395.1.1099_U;HM103418.1.1104_U;JN811207.1...,HM103395;HM103418;JN811207;GU825020;HM103399;H...,"Quaiser,A.. Comparative metagenomics of bathyp...",eSNCM,endosymbionts,"Decelle J, Siano R, Probert I et al (2012b) Mu...",no,not recorded,not recorded,not recorded,100,150,not recorded,micro,not applicable,"Decelle J, Siano R, Probert I et al (2012b) Mu...",not recorded,this work,prey capture using axopodia; engulfment after ...,pico-micro,not recorded,not recorded,not recorded,not recorded,not recorded,not applicable,not applicable,not applicable,not applicable,not applicable,not applicable,"Swanberg, N.R. and Caron, D.A., 1991. Patterns...","tintinnids, mollusc larvae, copepods, ostracod...","Diatomeae, Ciliophora, Copepoda, Mollusca, Ost...","Swanberg, N.R. and Harbison, G.R., 1980. The e...",not applicable,not applicable,not applicable,not applicable,"Chrysochromulina, Azadinium sp., Pelagodinium ...","Haptophyta, Dinoflagellata","Decelle J, Siano R, Probert I et al (2012b) Mu...",not recorded,4-15 um,nano,5 (<100 μm); 10-29 (100-200 μm); 29-45 (200-30...,"Decelle J, Siano R, Probert I et al (2012b) Mu...",-,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded,not recorded
1,Acanthometra fusca,Radiolaria,not registered,Acantharia,18S_rRNA_nucleus;18S_rRNA_nucleus;18S_rRNA_nuc...,KC172856.1.1696_U;EU446351.1.1552_U;JN811165.1...,KC172856;EU446351;JN811165,"Decelle,J.. Diversity, ecology and biogeochemi...",eSNCM,endosymbionts,"Michaels, A.F., 1988. Vertical distribution an...",no,not recorded,not recorded,not recorded,100-300,100-300,not recorded,micro-meso,not applicable,"Michaels, A.F., 1988. Vertical distribution an...",not recorded,this work,prey capture using axopodia; engulfment after ...,pico-micro,not recorded,not recorded,not recorded,not recorded,not recorded

In [62]:
csv_name = "LIS_2019-Phytoplankton_Final Report Data.xlsx - 2019 LIS phytoplankton count"

In [63]:
# import and clean LIS data
lis = pd.read_csv(f"inputs/{csv_name}.csv")
original_headers = lis.columns  # save original column headers

phylum_ind = lis[lis.iloc[:, 0] == "Phylum"].index[0]
lis.columns = lis.iloc[phylum_ind]  # reset column headers
lis = lis.iloc[phylum_ind+2:].reset_index(drop=True)  

In [64]:
# remove rows after unknown flagellates
unknown_flagellates_ind = lis[lis["Phylum"] == "Unknown flagellates"].index[0] 
lis = lis.iloc[:unknown_flagellates_ind]
lis = lis.iloc[:lis.last_valid_index()+1]  # remove trailing nan rows

In [65]:
# remove rows that contain "TOTAL"
lis = lis[~lis["Phylum"].str.contains("TOTAL", na=False)].reset_index(drop=True)  

In [66]:
# construct correct phylum column
actual_phylum_ind = lis[lis["Species"].isna() & lis["Phylum"].isna()].index + 1
lis = lis.rename(columns={"Phylum": "Genus"}) # rename phylum column to genus
lis.insert(0, 'Phylum', lis["Genus"].iloc[actual_phylum_ind])  # reconstruct phylum column
lis['Phylum'] = lis['Phylum'].ffill()  # forwardfill phylum

lis['Genus'] = lis['Species'].str.split().str[0]  # fill genus using first word of species name

lis = lis.dropna(subset=['Species']).reset_index(drop=True) # delete rows with na in Species column

SPECIES_COL = lis.columns.get_loc("Species")
lis.iloc[:, SPECIES_COL+1:] = lis.iloc[:, SPECIES_COL+1:].replace(",", "", regex=True).astype(float)  # ensure numerical values are floats

In [67]:
# add Status column
lis.insert(0, 'Status', None)

In [68]:
# store blocks of known mixotroph genuses 
ochromonas_ind = lis[lis["Species"].str.contains("Ochromonas")].index
ochromonas_block = lis.iloc[ochromonas_ind] 

In [69]:
# remove based on hard coded rules (NOT RESETTING INDEX IN ORDER TO ADD BLOCKS BACK CORRECTLY)
lis = lis[lis["Phylum"] != "Diatom"] # remove all diatoms
lis = lis[~lis["Species"].str.contains("-like")] # remove species ending with "-like"
lis = lis[~lis["Species"].str.contains("sp.|spp.")]  # remove all sp. / spp.

In [70]:
# check "cysts of"
CYSTS_LEN = len("cysts of ")
cysts_of = lis[lis["Species"].str.contains("cysts of", regex=False)]["Species"].str.slice(CYSTS_LEN)
filtered = cysts_of.isin(mdb['Species Name'])
lis.loc[filtered[filtered].index, "Status"] = "Confirmed"
lis.loc[filtered[filtered].index, "Genus"] = cysts_of.str.split().str[0]

In [71]:
# add back stored blocks of known mixotrophs and mark as Confirmed
lis = pd.concat([lis, ochromonas_block]).sort_index().drop_duplicates()
lis.loc[ochromonas_ind, "Status"] = "Confirmed"

In [72]:
# check if (in none status) direct match and mark all Trues as "Confirmed"
filtered = lis["Species"].isin(mdb['Species Name'])
lis.loc[filtered[filtered].index, "Status"] = "Confirmed"

# check (in remaining none status) if the genus has sp. and mark all Trues as "Unsure (sp. in mdb)"
genus_to_check = lis[lis['Status'].isnull()]['Species'].str.split().str[0].drop_duplicates() + " sp."
filtered = genus_to_check.isin(mdb['Species Name'])
lis.loc[filtered[filtered].index, "Status"] = "Unsure (sp. mdb)"

In [73]:
# check (in remaining none status) if the name is contained in the mdb and vice versa and mark all Trues as "Unsure (inexact name)"
filtered = lis[lis['Status'].isnull()]["Species"].apply(lambda x: mdb["Species Name"].str.contains(x, regex=False).any())
lis.loc[filtered[filtered].index, "Status"] = "Unsure (inexact name)"

pattern = '|'.join(mdb['Species Name'])
filtered = lis[lis['Status'].isnull()]["Species"].str.contains(pattern, regex=True)
lis.loc[filtered[filtered].index, "Status"] = "Unsure (inexact name)"

In [74]:
# drop all rows with Status = "None"
lis = lis.dropna(subset=['Status']).reset_index(drop=True)

In [75]:
lis = pd.merge(lis, mdb[['Species Name','MFT', 'Evidence of mixoplankton activity', 'size class']], left_on='Species', right_on='Species Name', how='left').drop(columns=['Species Name']).reset_index(drop=True) 


In [79]:
lis.shape

(16, 247)

In [81]:
# lis[lis.columns[:4].to_list() + lis.columns[-3:].to_list() + lis.columns[4:-3].to_list()]

In [86]:
len(lis.columns[4:-3].to_list())

240

In [88]:
lis.columns[:4].to_list() + lis.columns[-3:].to_list() + lis.columns[4:-3].to_list()

247

In [91]:
pd.concat([lis.iloc[:, :4], lis.iloc[:, -3:], lis.iloc[:, 4:-3]], axis=1).shape

(16, 247)

In [89]:
lis.reindex(columns=(lis.columns[:4].to_list() + lis.columns[-3:].to_list() + lis.columns[4:-3].to_list()))

ValueError: cannot reindex on an axis with duplicate labels

In [54]:
# add certain columns from mdb database
lis = pd.merge(lis, mdb[['Species Name','MFT', 'Evidence of mixoplankton activity', 'size class']], left_on='Species', right_on='Species Name', how='left').drop(columns=['Species Name']).reset_index(drop=True) 
lis = lis[lis.columns[:4].to_list() + lis.columns[-3:].to_list() + lis.columns[4:-3].to_list()]
lis[['MFT', 'Evidence of mixoplankton activity', 'size class']] = lis[['MFT', 'Evidence of mixoplankton activity', 'size class']].fillna("")

In [55]:
totals = lis.groupby('Phylum', as_index=False, sort=False).sum()

# empty text-containing columns
totals = totals.drop("Status", axis=1)
totals.insert(0, 'Status', "") 
totals["Genus"] = ""
totals["Species"] = ""
totals["MFT"] = ""
totals["Evidence of mixoplankton activity"] = ""
totals["size class"] = ""

# rename to TOTAL "   "
totals["Phylum"] = totals["Phylum"].str.upper().apply(lambda x: "TOTAL " + x + "S")

# add in line skips
totals = totals.set_index(lis.groupby(['Phylum']).tail(1).index + 0.1)
empty_df = pd.DataFrame("", index=lis.groupby(['Phylum']).tail(1).index+0.2, columns=totals.columns)
totals = pd.concat([totals, empty_df]).sort_index()

/var/folders/8f/krhzcyxd6c995sh8b6qrxymh0000gn/T/ipykernel_38753/84878263.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  totals = lis.groupby('Phylum', as_index=False, sort=False).sum()


1,Status,Phylum,Genus,Species,MFT,Evidence of mixoplankton activity,size class,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/3/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/7/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,1/2/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/6/19,2/6/19,2/6/19,2/6/19,2/6/19,2/6/19,2/6/19,2/6/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/4/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/7/19,2/6/19,2/6/19,2/6/19,...,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/6/19,11/6/19,11/6/19,11/6/19,11/6/19,11/6/19,11/6/19,11/6/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/5/19,11/6/19,11/6/19,11/6/19,11/6/19,11/6/19,11/6/19,11/6/19,11/6/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,11/4/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/5/19,12/5/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/6/19,12/5/19,12/5/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/16/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19,12/4/19
10.1,,TOTAL DINOFLAGELLATES,,,,,,17952.0,17600.0,30800.0,8800.0,2904.0,0,17952.0,17600.0,30800.0,8800.0,2904.0,0,17952.0,17600.0,30800.0,8800.0,2904.0,0,8800.0,0,17600.0,13200.0,14

In [56]:
lis = pd.concat([lis, totals]).sort_index().reset_index(drop=True)

In [57]:
# replace Nans with zero
lis = lis.fillna(0)

In [58]:
# add back multiheader
needed_cols = pd.Series(np.full(len(lis.columns) - len(original_headers), None)) 
original_headers = pd.concat([needed_cols, original_headers.to_series()], ignore_index=True)
lis.columns = pd.MultiIndex.from_arrays([original_headers, lis.columns])

In [59]:
lis


NaN                                        \
1                  Status                                Phylum   
0               Confirmed                        Dinoflagellate   
1               Confirmed                        Dinoflagellate   
2               Confirmed                        Dinoflagellate   
3               Confirmed                        Dinoflagellate   
4               Confirmed                        Dinoflagellate   
5               Confirmed                        Dinoflagellate   
6               Confirmed                        Dinoflagellate   
7               Confirmed                        Dinoflagellate   
8               Confirmed                        Dinoflagellate   
9               Confirmed                        Dinoflagellate   
10              Confirmed                        Dinoflagellate   
11                                        TOTAL DINOFLAGELLATES   
12                                                                
13  Unsure (inexact name)                          Raphidophyte   
14              Confirmed                          Raphidophyte   
15                                          TOTAL RAPHIDOPHYTES   
16                                                                
17              Confirmed         Ochrophyte (diatoms excluded)   
18              Confirmed         Ochrophyte (diatoms excluded)   
19                         TOTAL OCHROPHYTE (DIATOMS EXCLUDED)S   
20                                                                
21              Confirmed                            Haptophyte   
22                                            TOTAL HAPTOPHYTES   
23                                                                

                                                            \
1           Genus                          Species     MFT   
0        Akashiwo               Akashiwo sanguinea      CM   
1      Dinophysis             Dinophysis acuminata   pSNCM   
2      Dinophysis                 Dinophysis miles  pSNCM*   
3      Dinophysis             Dinophysis norvegica   pSNCM   
4   Gambierdiscus            Gambierdiscus toxicus      CM   
5       Gonyaulax             Gonyaulax polygramma      CM   
6     Heterocapsa      Heterocapsa circularisquama      CM   
7       Noctiluca            Noctiluca scintillans   eSNCM   
8    Prorocentrum                Prorocentrum lima      CM   
9    Prorocentrum              Prorocentrum micans      CM   
10  Lingulodinium  cysts of Lingulodinium polyedra           
11                                                           
12                                                           
13    Chattonella               Chattonella marina           
14    Heterosigma             Heterosigma akashiwo      CM   
15                                                           
16                                                           
17     Ochromonas                  Ochromonas spp.           
18     Ochromonas                Ochromonas danica           
19                                                           
20                                                           
21    Phaeocystis              Phaeocystis globosa      CM   
22                                                           
23                                                           

                                                                           \
1                   Evidence of mixoplankton activity size class   1/3/19   
0   ingestion of ciliates, Isochrysis, Cryptophyte...      micro        0   
1   photosynthetic Dinophysis spp. obtain plastids...      micro        0   
2   This species retains chloroplasts from cryptop...      micro        0   
3   photosynthetic Dinophysis spp. obtain plastids...      micro        0   
4          presence of feeding vacuoles, unknown prey      micro    352.0   
5   ingestion of cryptophyte species, Amphidinium ...      micro        0   
6                           bacteria in food vacuo

In [41]:
# save dataframe to excel
lis.to_excel(f"outputs/{csv_name}-{str(datetime.now())}.xlsx")